---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [3]:
# degrees = P1_Graphs[0].degree()
# sorted(degrees, key= lambda x: degrees[x],reverse=True)

# 'PA','SW_H','SW_L','PA','SW_H'

# for i,graph in enumerate(P1_Graphs):
# graph = P1_Graphs[3]
# degrees = graph.degree()
# print("# ndoes: ", len(graph.nodes()))
# print("Shortest path: ", nx.average_shortest_path_length(graph))
# print("Clustering coeff: ", sum(nx.clustering(graph).values())/len(graph.nodes()))
# degree_values = sorted(set(degrees.values()))
# dist=[list(degrees.values()).count(val) for val in degree_values]


# print(degree_values)
# print(dist)
# degrees

In [4]:
# list(degrees.values()).count


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']

In [6]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
def salary_predictions():
    
    dfGraph = pd.DataFrame(index=G.nodes())
    dfGraph["Department"] = pd.Series(nx.get_node_attributes(G, 'Department'))
    dfGraph["ManagementSalary"] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    dfGraph['clustering'] = pd.Series(nx.clustering(G))
    dfGraph['degree'] = pd.Series(G.degree())
    dfGraph['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    dfGraph['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    dfGraph['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    dfGraph['pr'] = pd.Series(nx.pagerank(G))
    # dfGraph.head()

    trainDs = dfGraph[~dfGraph["ManagementSalary"].isnull()]
    testDs = dfGraph[dfGraph["ManagementSalary"].isnull()]
    # trainDs.columns

    columns = ['Department', 'clustering', 'degree',
           'degree_centrality', 'closeness', 'betweeness', 'pr']

    X = trainDs[columns]
    # X
    Xtestds = testDs[columns]
    Y = trainDs["ManagementSalary"].reshape(-1, 1)
    Y
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y)
    Xtrain.shape
    Ytrain.shape


    gbclf = GradientBoostingClassifier().fit(Xtrain, Ytrain)
    y_score_gb_tr = gbclf.predict_proba(Xtrain)
    y_score_gb = gbclf.predict_proba(Xtest)
    y_score_gb_tr
    # print("auc score train tree: ",roc_auc_score( Ytrain,y_score_gb_tr[:,1]))
    # print("auc score test tree: ",roc_auc_score(Ytest,y_score_gb[:,1]))
    y_score_gb = gbclf.predict_proba(Xtestds)
    return  pd.Series(y_score_gb[:,1],Xtestds.index)


In [9]:
salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1       0.005530
2       0.984661
5       0.993122
8       0.187120
14      0.027276
18      0.013808
27      0.030953
30      0.887716
31      0.123939
34      0.018308
37      0.020460
40      0.019457
45      0.005264
54      0.131316
55      0.261225
60      0.081444
62      0.993122
65      0.987064
77      0.014245
79      0.015435
97      0.011363
101     0.003348
103     0.819639
108     0.007635
113     0.031525
122     0.001585
141     0.148987
142     0.993122
144     0.030724
145     0.889871
          ...   
913     0.014134
914     0.010965
915     0.001521
918     0.027856
923     0.010422
926     0.028283
931     0.006952
934     0.000766
939     0.001147
944     0.000895
945     0.010422
947     0.013013
950     0.018430
951     0.005830
953     0.003193
959     0.001147
962     0.001147
963     0.039776
968     0.014134
969     0.014134
974     0.013013
984     0.000766
987     0.011662
989     0.013013
991     0.025095
992     0.001147
994     0.001085
996     0.0012

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [10]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [11]:
def new_connections_predictions():
    
    
    
    
#     for node in G.nodes():
#         G.node[node]['community'] = G.node[node]['Department']
        
#     df['commonNScore'] = [item[2] for item in list(nx.cn_soundarajan_hopcroft(G))]    
    
    
    
    future_connections['commonN'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['prefAtt'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]    
    future_connections['rai'] = [item[2] for item in list(nx.resource_allocation_index(G, future_connections.index))]
    future_connections['jaccardCoeff'] = [item[2] for item in list(nx.jaccard_coefficient(G, future_connections.index))]

    trainDs = future_connections[~pd.isnull(future_connections['Future Connection'])]
    testDs = future_connections[pd.isnull(future_connections['Future Connection'])]

    columns = ['commonN', 'prefAtt', 'rai', 'jaccardCoeff']
    X = trainDs[columns]
    Y = trainDs['Future Connection']
    Xtestds = testDs[columns]

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y)
    Xtrain.shape
    Xtestds.shape

    gbclf = GradientBoostingClassifier().fit(Xtrain, Ytrain)
    y_score_gb_tr = gbclf.predict_proba(Xtrain)
    y_score_gb = gbclf.predict_proba(Xtest)
    y_score_gb_tr
    # print("auc score train tree: ",roc_auc_score( Ytrain,y_score_gb_tr[:,1]))
    # print("auc score test tree: ",roc_auc_score(Ytest,y_score_gb[:,1]))
    y_score_gb = gbclf.predict_proba(Xtestds)
    return pd.Series(y_score_gb[:,1],Xtestds.index)

In [12]:
result = new_connections_predictions()

In [13]:
result

(107, 348)    0.026809
(542, 751)    0.013122
(20, 426)     0.565242
(50, 989)     0.013122
(942, 986)    0.013122
(324, 857)    0.013122
(13, 710)     0.112534
(19, 271)     0.148546
(319, 878)    0.013122
(659, 707)    0.013122
(49, 843)     0.013122
(208, 893)    0.013122
(377, 469)    0.013740
(405, 999)    0.018159
(129, 740)    0.013122
(292, 618)    0.060312
(239, 689)    0.013122
(359, 373)    0.013698
(53, 523)     0.030442
(276, 984)    0.013122
(202, 997)    0.013122
(604, 619)    0.069754
(270, 911)    0.013122
(261, 481)    0.058529
(200, 450)    0.904293
(213, 634)    0.013122
(644, 735)    0.147155
(346, 553)    0.013122
(521, 738)    0.013122
(422, 953)    0.017890
                ...   
(672, 848)    0.013122
(28, 127)     0.973177
(202, 661)    0.013122
(54, 195)     0.998062
(295, 864)    0.013122
(814, 936)    0.013122
(839, 874)    0.013122
(139, 843)    0.013122
(461, 544)    0.013122
(68, 487)     0.013122
(622, 932)    0.013122
(504, 936)    0.015766
(479, 528) 